In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, r'C:/Users/Angie Michaiel/Documents/GitHub/FreelyMovingEphys')
from util.ephys import ephys_to_dataset
from util.paths import find
from util.ephys import ephys_to_dataframe

In [3]:
#Paths for diffrent recording based on condition
recordings_path = r'\\kraken\t\freely_moving_ephys\ephys_recordings'
conditions = {}
conditions['dates_pathsaline'] = ['021121']
conditions['dates_pathdoi'] = ['021021']
dates_path = [val for sublists in conditions.values() for val in sublists]
print(conditions)
print(dates_path)



{'dates_pathsaline': ['021121'], 'dates_pathdoi': ['021021']}
['021121', '021021']


In [4]:
conditions.get('date')

In [5]:
collected_ephys = ephys_to_dataframe(recordings_path,dates_path,conditions)

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
collected_ephys

In [ ]:
collected_ephys.dtypes


In [ ]:
collected_ephys.columns


In [ ]:
collected_ephys.date

In [ ]:
collected_ephys.doi